# Setup

### Libraries

In [ ]:
import pandas as pd
import numpy as np

import cmdstanpy
from cmdstanpy import cmdstan_path, CmdStanModel

#cmdstanpy.install_cmdstan()

### Data

In [ ]:
#Load and prepare the covariates, location dummies and housing prices
#For both training and test sets
housing_training_df = pd.read_csv('../data/training_set.csv')
housing_training_df.dropna(inplace = True)

housing_test_df = pd.read_csv('../data/test_set.csv')
housing_test_df.dropna(inplace = True)
#-----------------------------------------------------------------------------------------

#Y
#-----------------------------------------------------------------------------------------
housing_prices_training = housing_training_df['median_hou']
housing_prices_test =     housing_test_df['median_hou'] 
#-----------------------------------------------------------------------------------------

#X
#-----------------------------------------------------------------------------------------
covariate_columns = ['avg_room', 'avg_bedr', 'population',
                     'households', 'median_inc', 'housing_me'] #'ocean_prox'
        
housing_covariates_training = housing_training_df[covariate_columns]
housing_covariates_test =     housing_test_df[covariate_columns] 

#Non-spatial X
non_spatial_covariates = ['avg_bedr', 'population', 'households',
                          'median_inc', 'housing_me']

housing_non_spatial_covariates_training = housing_training_df[non_spatial_covariates]
housing_non_spatial_covariates_test = housing_test_df[non_spatial_covariates]

#Spatial X
spatial_covariates = ['avg_room']

housing_spatial_covariates_training = housing_training_df[spatial_covariates]
housing_spatial_covariates_test = housing_test_df[spatial_covariates]
#-----------------------------------------------------------------------------------------

#Location dummies
#-----------------------------------------------------------------------------------------
location_dummies_training = housing_training_df.loc[:, "Alameda":"Yuba":1]
location_dummies_test =     housing_test_df.loc[:, "Alameda":"Yuba":1]

county_idx_training = np.argmax(location_dummies_training.to_numpy(), axis=1) + 1
county_idx_test = np.argmax(location_dummies_test.to_numpy(), axis=1) + 1
#-----------------------------------------------------------------------------------------

#Ocean dummies
#-----------------------------------------------------------------------------------------
ocean_dummies_training = housing_training_df.loc[:,"<1H OCEAN":"NEAR OCEAN":1]
ocean_dummies_test =     housing_test_df.loc[:,"<1H OCEAN":"NEAR OCEAN":1]
#-----------------------------------------------------------------------------------------

#Turn the data into numpy
#-----------------------------------------------------------------------------------------
ocean_dummies_training = ocean_dummies_training.to_numpy()
location_dummies_training = location_dummies_training.to_numpy()
housing_covariates_training = housing_covariates_training.to_numpy()
housing_prices_training = housing_prices_training.to_numpy()

ocean_dummies_test = ocean_dummies_test.to_numpy()
location_dummies_test = location_dummies_test.to_numpy()
housing_covariates_test = housing_covariates_test.to_numpy()
housing_prices_test = housing_prices_test.to_numpy()

housing_non_spatial_covariates_training = housing_non_spatial_covariates_training.to_numpy()
housing_spatial_covariates_training = housing_spatial_covariates_training.to_numpy()

housing_spatial_covariates_training = housing_spatial_covariates_training.reshape(
    (housing_spatial_covariates_training.shape[0],)
)

housing_non_spatial_covariates_test = housing_non_spatial_covariates_test.to_numpy()
housing_spatial_covariates_test = housing_spatial_covariates_test.to_numpy()

housing_spatial_covariates_test = housing_spatial_covariates_test.reshape(
    (housing_spatial_covariates_test.shape[0],)
)

#Have to make sure that no housing price = 0, therefore add a small buffer
housing_prices_training += 10**(-7) #+ np.abs(housing_prices_training.min())
housing_prices_test += 10**(-7) #+ np.abs(housing_prices_test.min())

def unnormalize(prices):
  return prices * 484101.0 + 14999.0

#Load proximity_matrix from Utils
proximity_matrix = pd.read_csv('../data/W_kernel.csv', delimiter=',')
proximity_matrix = proximity_matrix.to_numpy()

"""# to make the prior on spatial dependence proper we introduce the spatial dependence parameter e
e = 0.99
proximity_matrix = neighbour_matrix * e"""
print(proximity_matrix.shape)

### Priors

In [ ]:
#Read in the raw data
classic_raw_priors = pd.read_csv('../data/classic_mean_and_std.csv')
hierarchichal_raw_priors = pd.read_csv('../data/hierarchical_mean_and_std.csv')
simple_CAR_raw_priors = pd.read_csv('../data/simple_CAR_mean_and_std.csv')
CAR_raw_priors = pd.read_csv('../data/CAR_mean_and_std_dev.csv')

#Separate the parameters, and get the necessary values

#Classical
#----------------------------------------------------------------------------
classic_regr_params_means = classic_raw_priors['Mean'][0:6].to_numpy()
classic_regr_params_std = classic_raw_priors['StdDev'][0:6].to_numpy()
classic_regr_params_variance = classic_regr_params_std**2

print("classic_regr_params_std: ", classic_regr_params_std)
print("classic_regr_params_variance: ", classic_regr_params_variance)


classic_ocean_params_means = classic_raw_priors['Mean'][6:11].to_numpy()
classic_ocean_params_std = classic_raw_priors['StdDev'][6:11].to_numpy()
classic_ocean_params_variance = classic_ocean_params_std**2

print("classic_ocean_params_std: ", classic_ocean_params_std)
print("classic_ocean_params_variance: ", classic_ocean_params_variance)

classic_housing_prices_variance_means = classic_raw_priors['Mean'][11]
classic_housing_prices_variance_std = classic_raw_priors['StdDev'][11]
classic_housing_prices_variance_variance = classic_housing_prices_variance_std**2

print("classic_housing_prices_variance_std: ", classic_housing_prices_variance_std)
print("classic_housing_prices_variance_variance: ", classic_housing_prices_variance_variance)
#----------------------------------------------------------------------------

#Hiearchichal
#----------------------------------------------------------------------------
print("---")
#Gamma0
gamma_0 = hierarchichal_raw_priors['Mean'][0:6].to_numpy()
print("gamma_0: ", gamma_0)

#L0
L = np.zeros((6,6))
for i in range(6):
  for j in range(6):
    L[i,j] = hierarchichal_raw_priors['Mean'][
        (11 + i*6 + j):(11 + i*6 + j + 1)]

print("L: ", L)

#M0
M = np.zeros((5,5))
for i in range(5):
  for j in range(5):
    M[i,j] = hierarchichal_raw_priors['Mean'][(47 + i*5 + j):(47 + i*5 + j + 1)]

print("M: ", M)

#Sigma0
Sigma = np.zeros((6,6))
for i in range(6):
  for j in range(6):
    Sigma[i, j] = hierarchichal_raw_priors['Mean'][(72 + i*6 + j):(72 + i*6 + j + 1)]

print("SIGMA: ", Sigma)

#sigma0 and nu0
sigma_0_hierarchical = hierarchichal_raw_priors['Mean'][(47+25+36):(47+25+36+1)].to_numpy()
nu_0_hierarchical = hierarchichal_raw_priors['StdDev'][(47+25+36):(47+25+36+1)].to_numpy()

print("sigma_0: ", sigma_0_hierarchical)
print("nu_0: ", nu_0_hierarchical)

#Delta
delta_mean = hierarchichal_raw_priors['Mean'][(6):(6+5)].to_numpy()

print("---")
#----------------------------------------------------------------------------

#Simple CAR
#----------------------------------------------------------------------------
simple_CAR_regr_params = simple_CAR_raw_priors[0:6]
remaining_interesting_simple_Car = simple_CAR_raw_priors[(6+58):len(simple_CAR_raw_priors)]
ocean_params_CAR = remaining_interesting_simple_Car[0:5]
housing_prices_variance_CAR = remaining_interesting_simple_Car[5:6]
phi_variance_CAR = remaining_interesting_simple_Car[6:7]
e_CAR = remaining_interesting_simple_Car[7:8]

#---
simple_CAR_regr_params_means = simple_CAR_regr_params['Mean'].to_numpy()
simple_CAR_regr_params_std = simple_CAR_regr_params['StdDev'].to_numpy()
simple_CAR_regr_params_variance = simple_CAR_regr_params_std**2


ocean_params_CAR_means = ocean_params_CAR['Mean'].to_numpy()
ocean_params_CAR_std = ocean_params_CAR['StdDev'].to_numpy()
ocean_params_CAR_variance = ocean_params_CAR_std**2

housing_prices_variance_CAR_means = (housing_prices_variance_CAR['Mean'].to_numpy())[0]
housing_prices_variance_CAR_std = (housing_prices_variance_CAR['StdDev'].to_numpy())[0]
housing_prices_variance_CAR_variance = housing_prices_variance_CAR_std**2

phi_variance_CAR_means = (phi_variance_CAR['Mean'].to_numpy())[0]
phi_variance_CAR_std = (phi_variance_CAR['StdDev'].to_numpy())[0]
phi_variance_CAR_variance = phi_variance_CAR_std**2

e_CAR_means = (e_CAR['Mean'].to_numpy())[0]

#There is a lot of drift in e, which causes drift in phi and ocean parameters, 
#reduce the variance to cope. Found 0.15 through trial and error (maybe 0.2 is better).
e_CAR_variance = ((e_CAR['StdDev'].to_numpy())[0])**2 * 0.15

#In order to get alpha_e and beta_e, must solve
#alpha_e/(alpha_e+beta_e)=e_CAR_means and
#(alpha_e*beta_e)/((alpha_e+beta_e)^2*(alpha_e+beta_e+1))=e_CAR_variance

from scipy.optimize import fsolve
import math

def equations(p):
    alpha_e, beta_e = p
    return (alpha_e/(alpha_e + beta_e) - e_CAR_means,
            (alpha_e * beta_e)/((alpha_e + beta_e)**2 * (alpha_e + beta_e + 1)) - e_CAR_variance)

alpha_e, beta_e =  fsolve(equations, (1/2, 1/2))
print("e_CAR_means: ", e_CAR_means)
print("e_CAR_variance: ", e_CAR_variance)
print(equations((alpha_e, beta_e)))
print("alpha_e: ", alpha_e)
print("beta_e: ", beta_e)
#----------------------------------------------------------------------------

#CAR model
#----------------------------------------------------------------------------
non_spatial_regr_means = CAR_raw_priors[1:6]['Mean'].to_numpy()
non_spatial_regr_std = CAR_raw_priors[1:6]['StdDev'].to_numpy()

avg_room_std = CAR_raw_priors[6:(6+58)]['StdDev'].to_numpy()

car_ocean_means = CAR_raw_priors[122:127]['Mean'].to_numpy()
car_ocean_std = CAR_raw_priors[122:127]['StdDev'].to_numpy()

car_housing_prices_variance_means = (CAR_raw_priors[127:128]['Mean'].to_numpy())[0]
car_housing_prices_variance_std = (CAR_raw_priors[127:128]['StdDev'].to_numpy())[0]

car_phi_variance_means = (CAR_raw_priors[128:129]['Mean'].to_numpy())[0]
car_phi_variance_std = (CAR_raw_priors[128:129]['StdDev'].to_numpy())[0]

e_phi_mean = (CAR_raw_priors[129:130]['Mean'].to_numpy())[0]
e_phi_std = ((CAR_raw_priors[129:130]['StdDev'].to_numpy())[0])**2

def equations(p):
    alpha_e, beta_e = p
    return (alpha_e/(alpha_e + beta_e) - e_phi_mean,
            (alpha_e * beta_e)/((alpha_e + beta_e)**2 * (alpha_e + beta_e + 1)) - e_phi_std)

alpha_e_phi, beta_e_phi =  fsolve(equations, (1/2, 1/2))

e_avg_room_mean = (CAR_raw_priors[130:131]['Mean'].to_numpy())[0]
e_avg_room_std = ((CAR_raw_priors[130:131]['StdDev'].to_numpy())[0])**2

def equations(p):
    alpha_e, beta_e = p
    return (alpha_e/(alpha_e + beta_e) - e_avg_room_mean,
            (alpha_e * beta_e)/((alpha_e + beta_e)**2 * (alpha_e + beta_e + 1)) - e_avg_room_std)

alpha_e_avg_room, beta_e_avg_room =  fsolve(equations, (1/2, 1/2))

# Run the models (and optionally, save)

## Simple GLM

#### Define

In [ ]:
classic_reg = """
data {
  int<lower = 0>                 N_data;
  int<lower = 0>                 N_features;
  int<lower = 0, upper = 1>      withOcean;
  int<lower = 0>                 N_ocean;

  vector[N_data]                 house_prices;
  matrix[N_data, N_features]     X;
  matrix[N_data, N_ocean]        ocean_dummies;

  real prior_regr_parameters_mean[N_features];
  real prior_regr_parameters_std[N_features];

  real prior_ocean_params_mean[N_ocean];
  real prior_ocean_params_std[N_ocean];

  real prior_housing_prices_variance_expectation;
  real prior_housing_prices_variance_std;  
}

parameters {
  vector[N_features]      regression_parameters;
  vector[N_ocean]         ocean_parameters;

  real<lower = 0.00001>   housing_prices_variance;
}

transformed parameters {
  vector<lower=0>[N_data] mu;
  vector<lower=0>[N_data] beta;                        
  vector<lower=0>[N_data] alpha;      

  mu = exp(X * regression_parameters + withOcean * ocean_dummies * ocean_parameters);
  beta = mu / housing_prices_variance;
  alpha = mu .* beta;       
}

model {
  house_prices ~ gamma(alpha, beta);

  regression_parameters ~ normal(prior_regr_parameters_mean, prior_regr_parameters_std);
  ocean_parameters ~ normal(prior_ocean_params_mean, prior_ocean_params_std);

  housing_prices_variance ~ normal(prior_housing_prices_variance_expectation, 
                                   prior_housing_prices_variance_std);
}

generated quantities  {
  vector[N_data] y_rep;
  for(n in 1:N_data){
    y_rep[n] = gamma_rng(alpha[n],beta[n]); //posterior draws to get posterior predictive checks
  }

  vector[N_data] log_lik; // To compute Waic measure
  for (n in 1:N_data) log_lik[n] = gamma_lpdf(house_prices[n] | alpha[n], beta[n]);
}
"""

#### Compile

In [ ]:

stan_file = "./classic_reg.stan"

with open(stan_file, "w") as f:
    print(classic_reg, file=f)

classic_reg_model = CmdStanModel(stan_file=stan_file)

#### Run

In [ ]:
reg_data = {
    "N_data" : housing_prices_training.shape[0],
    "N_features": housing_covariates_training.shape[1],
    "withOcean" : 1,
    "N_ocean" : ocean_dummies_training.shape[1],
    "house_prices" : housing_prices_training,
    "X": housing_covariates_training,
    "ocean_dummies": ocean_dummies_training,
    "prior_regr_parameters_mean": classic_regr_params_means,
    "prior_regr_parameters_std": classic_regr_params_std,
    "prior_ocean_params_mean": classic_ocean_params_means,
    "prior_ocean_params_std": classic_ocean_params_std,
    "prior_housing_prices_variance_expectation": classic_housing_prices_variance_means,
    "prior_housing_prices_variance_std": classic_housing_prices_variance_std, 
}

gamma_fit = classic_reg_model.sample(reg_data, chains=4, parallel_chains=4, 
                             iter_warmup=1000, iter_sampling=1000)

#### Save (optional)

In [ ]:
save_directory = "../data/simple_GLM"
gamma_fit.save_csvfiles(save_directory)

## Hierarchical

#### Define

In [ ]:
hiergamma_reg = """
    data {
      int<lower=0> n_group;   // number of groups
      int<lower=0> N;   // number of data items
      int<lower=0> K;   // number of predictors (with intercept) for counties effects
      int<lower=0> P;   // number of predictors for (only) fixed effects
      matrix[N, K] x;   // predictor matrix counties effects
      matrix[N, P] z;   // predictor matrix fixed effects
      int<lower=1, upper=n_group> idgroup[N]; // specifies the county of the datapoint
      vector[N] y;      // outcome vector
      int<lower=0, upper=1> withOcean; //0 without, 1 with
      
      vector[K] prior_gamma_0;
      vector[P] prior_delta_0;
      matrix[K, K] prior_L;
      matrix[P, P] prior_M;
      matrix[K, K] prior_Sigma;
      real<lower = 0.00001> sigma_0;
      real<lower = 0.00001> nu_0;
    }
    
    parameters {
      matrix[n_group, K] gamma;       // coefficients for predictors; depending on the county
      vector[K] theta;
      vector[P] delta;           // Coefficients for fixed predictors
      corr_matrix[K] L;        // Covariance matrix for beta
      corr_matrix[P] M;        // Covariance matrix for delta
      corr_matrix[K] Sigma;
      real<lower=0> sigma;
    }
        
    transformed parameters {
      vector[N] mu; // Mean of every house (only as output)
      vector[N] alpha; // Shape parameter
      vector[N] beta; //
       for (i in 1:N) {
          mu[i]=exp(x[i]*gamma[idgroup[i]]'+z[i]*delta); // Product of the characteristics of i-th house times the coefficient vector of county i
        }
        beta = mu/sigma; 
        for (i in 1:N){
          alpha[i]=mu[i]*beta[i];
        }

    }

    model {
    y ~ gamma(alpha, beta);
     for (i in 1:n_group) {
       gamma[i] ~ multi_normal(theta,L); // Non-informative Prior
     }
       theta ~ multi_normal(prior_gamma_0, Sigma);
       delta ~ multi_normal(prior_delta_0, M); // Non-informative Prior
       Sigma ~ inv_wishart(K-0.999, prior_Sigma);
       L ~ inv_wishart(K-0.999, prior_L); // Prior covariance matrix
       M ~ inv_wishart(P-0.999, prior_M); // Prior covariance matrix for fixed effects
       sigma ~ normal(sigma_0, nu_0); 
    }
"""

#### Compile

In [ ]:
stan_file = "./hiergamma.stan"

with open(stan_file, "w") as f:
    f.write(hiergamma_reg)

hiergamma_reg = CmdStanModel(stan_file=stan_file)

#### Run

In [ ]:
reg_data = {
    "n_group": proximity_matrix.shape[0],
    "N": housing_prices_training.shape[0],
    "K": housing_covariates_training.shape[1],
    "x": housing_covariates_training,
    "P": ocean_dummies_training.shape[1],
    "z": ocean_dummies_training,
    "idgroup" : county_idx_training,
    "y": housing_prices_training,
    "withOcean" : 1,
    "prior_gamma_0": gamma_0,
    "prior_delta_0": delta_mean,
    "prior_L": L,
    "prior_M": M,
    "prior_Sigma": Sigma,
    "sigma_0": sigma_0_hierarchical[0],
    "nu_0": nu_0_hierarchical[0],
}
 
gamma_fit = hiergamma_reg.sample(reg_data, chains=2, parallel_chains=2, 
                             iter_warmup=1000, iter_sampling=1000)

#### Save (optional)

In [ ]:
save_directory = "../data/hierarchical"
gamma_fit.save_csvfiles(save_directory)

## Simple CAR

#### Define

In [ ]:
simple_CAR = """
data {
  int<lower = 0>                 N_counties;
  int<lower = 0>                 N_data;
  int<lower = 0>                 N_features;
  int<lower = 0, upper = 1>      withOcean;
  int<lower = 0>                 N_ocean;

  vector[N_data]                 house_prices;
  matrix[N_data, N_features]     X;
  matrix[N_data, N_counties]     location_dummies;
  matrix[N_data, N_ocean]        ocean_dummies;
  matrix[N_counties, N_counties] proximity_matrix;

  real prior_regr_parameters_mean[N_features];
  real prior_regr_parameters_std[N_features];
  real prior_ocean_params_mean[N_ocean];
  real prior_ocean_params_std[N_ocean];
  real prior_housing_prices_variance_mean;
  real prior_housing_prices_variance_std;
  real prior_phi_variance_mean;
  real prior_phi_variance_std;
  real prior_alpha_e;
  real prior_beta_e;
}

parameters {
  vector[N_features]      regression_parameters;
  vector[N_counties]      phi;
  vector[N_ocean]         ocean_parameters;

  real<lower = 0.00001>   housing_prices_variance;
  real<lower = 0.00001>   phi_variance;
  real<lower = 0.001, upper = 0.999>     e;
}

transformed parameters {
  vector<lower=0>[N_data] mu;
  vector<lower=0>[N_data] beta;                        
  vector<lower=0>[N_data] alpha; 
  vector[N_counties]      phi_minus_i_weighted;        

  mu = exp(X * regression_parameters + location_dummies * phi + withOcean * ocean_dummies * ocean_parameters);
  beta = mu / housing_prices_variance;
  alpha = mu .* beta;     

  phi_minus_i_weighted = e * proximity_matrix * phi;         
}

model {
  house_prices ~ gamma(alpha, beta);

  regression_parameters ~ normal(prior_regr_parameters_mean, prior_regr_parameters_std);

  ocean_parameters ~ normal(prior_ocean_params_mean, prior_ocean_params_std);

  phi ~ normal(phi_minus_i_weighted, phi_variance);

  phi_variance ~ normal(prior_phi_variance_mean, prior_phi_variance_std);
  housing_prices_variance ~ normal(prior_housing_prices_variance_mean, 
                                   prior_housing_prices_variance_std);
  
  e ~ beta(prior_alpha_e , prior_beta_e);
}

generated quantities  {
  vector[N_data] y_rep;
  for(n in 1:N_data){
    y_rep[n] = gamma_rng(alpha[n],beta[n]); //posterior draws to get posterior predictive checks
  }

  vector[N_data] log_lik; // To compute Waic measure
  for (n in 1:N_data) log_lik[n] = gamma_lpdf(house_prices[n] | alpha[n], beta[n]);
}
"""

#### Compile

In [ ]:
stan_file = "./simple_CAR.stan"

with open(stan_file, "w") as f:
    print(simple_CAR, file=f)

simple_CAR_model = CmdStanModel(stan_file=stan_file)

#### Run

In [ ]:
reg_data = {
    "N_counties": proximity_matrix.shape[0],
    "N_data" : housing_prices_training.shape[0],
    "N_features": housing_covariates_training.shape[1],
    "withOcean" : 1,
    "N_ocean" : ocean_dummies_training.shape[1],
    "house_prices" : housing_prices_training,
    "X": housing_covariates_training,
    "location_dummies": location_dummies_training,
    "ocean_dummies": ocean_dummies_training,
    "proximity_matrix": proximity_matrix,
    "prior_regr_parameters_mean": simple_CAR_regr_params_means,
    "prior_regr_parameters_std": simple_CAR_regr_params_std,
    "prior_ocean_params_mean": ocean_params_CAR_means,
    "prior_ocean_params_std": ocean_params_CAR_std,
    "prior_housing_prices_variance_mean": housing_prices_variance_CAR_means,
    "prior_housing_prices_variance_std": housing_prices_variance_CAR_std,
    "prior_phi_variance_mean": phi_variance_CAR_means,
    "prior_phi_variance_std": phi_variance_CAR_std,
    "prior_alpha_e": alpha_e,
    "prior_beta_e": beta_e,
}

gamma_fit = simple_CAR_model.sample(reg_data, chains=2, parallel_chains=2, 
                             iter_warmup=1000, iter_sampling=1000)

#### Save (optional)

In [ ]:
save_directory = "../data/simple_CAR"
gamma_fit.save_csvfiles(save_directory)

## Full CAR

#### Define

In [ ]:
full_CAR = """
data {
  int<lower = 0>                 N_counties;
  int<lower = 0>                 N_data;
  int<lower = 0>                 N_non_spatial_features;
  int<lower = 0>                 N_spatial_features;
  int<lower = 0, upper = 1>      withOcean;
  int<lower = 0>                 N_ocean;
  
  vector[N_data]                             house_prices;
  vector[N_data]                             avg_room;
  matrix[N_data, N_non_spatial_features]     non_spatial_X;
  matrix[N_data, N_counties]                 location_dummies;
  matrix[N_data, N_ocean]                    ocean_dummies;
  matrix[N_counties, N_counties]             proximity_matrix;

  real prior_non_spatial_regr_parameters_mean[N_non_spatial_features];
  real prior_non_spatial_regr_parameters_std[N_non_spatial_features];
  real prior_ocean_params_mean[N_ocean];
  real prior_ocean_params_std[N_ocean];
  real prior_housing_prices_variance_mean;
  real prior_housing_prices_variance_std;
  real prior_phi_variance_mean;
  real prior_phi_variance_std;
  real prior_spatial_regr_parameter_std[N_counties];
  real prior_alpha_e_phi;
  real prior_beta_e_phi;
  real prior_alpha_e_avg_room;
  real prior_beta_e_avg_room;
}

parameters {
  vector[N_non_spatial_features]              non_spatial_regression_parameters;
  vector[N_counties]      avg_room_regression_parameters;
  vector[N_counties]      phi;
  vector[N_ocean]         ocean_parameters;

  real<lower = 0.00001>   housing_prices_variance;
  real<lower = 0.00001>   phi_variance;
  real<lower = 0, upper = 1>     e_phi;
  real<lower = 0, upper = 1>     e_avg_room;
}

transformed parameters {
  vector<lower=0>[N_data] mu;
  vector<lower=0>[N_data] beta;                        
  vector<lower=0>[N_data] alpha; 
  vector[N_counties]      phi_minus_i_weighted; 
  vector[N_counties]      avg_room_regr_param_minus_i_weighted;

  mu = exp(non_spatial_X * non_spatial_regression_parameters + 
           avg_room .* (location_dummies * avg_room_regression_parameters) +
           location_dummies * phi +
           withOcean * ocean_dummies * ocean_parameters);      
  
  beta = mu / housing_prices_variance;
  alpha = mu .* beta;     

  phi_minus_i_weighted = e_phi * proximity_matrix * phi;  
  avg_room_regr_param_minus_i_weighted = e_avg_room * proximity_matrix * avg_room_regression_parameters;      
}

model {
  house_prices ~ gamma(alpha, beta);

  non_spatial_regression_parameters ~ normal(prior_non_spatial_regr_parameters_mean, prior_non_spatial_regr_parameters_std);
  avg_room_regression_parameters ~ normal(avg_room_regr_param_minus_i_weighted, prior_spatial_regr_parameter_std);

  ocean_parameters ~ normal(prior_ocean_params_mean, prior_ocean_params_std);

  phi ~ normal(phi_minus_i_weighted, phi_variance);
  phi_variance ~ normal(prior_phi_variance_mean, prior_phi_variance_std);
  housing_prices_variance ~ normal(prior_housing_prices_variance_mean, prior_housing_prices_variance_std);
  
  e_phi ~ beta(prior_alpha_e_phi, prior_beta_e_phi);
  e_avg_room ~ beta(prior_alpha_e_avg_room, prior_beta_e_avg_room);
}

generated quantities  {
  vector[N_data] y_rep;
  for(n in 1:N_data){
    y_rep[n] = gamma_rng(alpha[n],beta[n]); //posterior draws to get posterior predictive checks
  }

  vector[N_data] log_lik; // To compute Waic measure
  for (n in 1:N_data) log_lik[n] = gamma_lpdf(house_prices[n] | alpha[n], beta[n]);
}
"""

#### Compile

In [ ]:
stan_file = "./full_CAR.stan"

with open(stan_file, "w") as f:
    print(full_CAR, file=f)

full_CAR_model = CmdStanModel(stan_file=stan_file)

#### Run

In [ ]:
reg_data = {
    "N_counties": proximity_matrix.shape[0],
    "N_data" : housing_prices_training.shape[0],
    "N_non_spatial_features": housing_non_spatial_covariates_training.shape[1],
    "N_spatial_features": 1,
    "withOcean" : 1,
    "N_ocean" : ocean_dummies_training.shape[1],
    "house_prices" : housing_prices_training,
    "avg_room": housing_spatial_covariates_training,
    "non_spatial_X": housing_non_spatial_covariates_training,
    "location_dummies": location_dummies_training,
    "ocean_dummies": ocean_dummies_training,
    "proximity_matrix": proximity_matrix,
    "prior_non_spatial_regr_parameters_mean": non_spatial_regr_means,
    "prior_non_spatial_regr_parameters_std": non_spatial_regr_std,
    "prior_ocean_params_mean": car_ocean_means,
    "prior_ocean_params_std": car_ocean_std,
    "prior_housing_prices_variance_mean": car_housing_prices_variance_means,
    "prior_housing_prices_variance_std": car_housing_prices_variance_std,
    "prior_phi_variance_mean": car_phi_variance_means,
    "prior_phi_variance_std": car_phi_variance_std,
    "prior_spatial_regr_parameter_std": avg_room_std,
    "prior_alpha_e_phi": alpha_e_phi,
    "prior_beta_e_phi": beta_e_phi,
    "prior_alpha_e_avg_room": alpha_e_avg_room,
    "prior_beta_e_avg_room": beta_e_avg_room,
}

CAR_fit = full_CAR_model.sample(reg_data, chains=1, parallel_chains=1, 
                             iter_warmup=1000, iter_sampling=1000)

#### Save (optional)

In [ ]:
save_directory = "../data/full_CAR"
gamma_fit.save_csvfiles(save_directory)